In [1]:
import pandas as pd
import numpy as np
import folium
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta
from tqdm import tqdm

In [2]:
fuca_01 = pd.read_csv("./data/fuca/fuca_01.csv")

In [3]:
fuca_01.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo
0,367479990,2017-01-01T00:00:03,48.15891,-122.67268,0.1,10.1,353.0,WSF KENNEWICK,IMO9618331,WDF6991,1012.0,moored,83.39,19.50,3.2,NaN
1,367585430,2017-01-01T00:00:12,48.51293,-122.60735,0.0,-101.2,511.0,MAKARIOS,NaN,WDG9205,1019.0,NaN,18.11,5.10,NaN,NaN
2,316021351,2017-01-01T00:00:00,49.29479,-123.13286,0.0,-118.3,511.0,BRITANNIA,IMO8120650,VY5348,1012.0,under way using engine,38.41,9.76,2.8,60.0
3,367536390,2017-01-01T00:00:39,48.68218,-123.40237,0.0,82.1,511.0,APRIL LOON II,NaN,WDG4420,1019.0,NaN,14.94,4.70,NaN,NaN
4,316005771,2017-01-01T00:00:47,48.95017,-125.55641,0.0,-94.6,511.0,WESTERN INVESTOR,IMO7927788,VG6273,1001.0,NaN,24.06,7.17,2.9,NaN


In [4]:
ves_type = list(sorted(fuca_01['VesselType'].unique()))

In [6]:
ves_type = list(map(int, ves_type[:-1]))

In [7]:
ves_type

[0,
 7,
 30,
 31,
 32,
 33,
 35,
 37,
 50,
 51,
 52,
 53,
 60,
 70,
 71,
 74,
 79,
 80,
 90,
 99,
 1001,
 1002,
 1003,
 1004,
 1005,
 1010,
 1011,
 1012,
 1013,
 1017,
 1018,
 1019,
 1020,
 1022,
 1023,
 1024,
 1025]

In [9]:
fuca_01[fuca_01['VesselType'].isna()]

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo
85,316026186,2017-01-01T00:00:56,49.15111,-122.99332,0.0,88.1,511.0,GREAT ESCAPE,NaN,NaN,NaN,NaN,24.40,NaN,NaN,NaN
152,316031048,2017-01-01T00:00:08,49.29074,-123.06070,0.0,-113.7,320.0,SST CAPILANO,IMO9744477,NaN,NaN,under way sailing,21.69,NaN,NaN,NaN
179,338072397,2017-01-01T00:02:58,48.00288,-122.22071,0.0,-110.3,511.0,ICEWEGIAN,NaN,NaN,NaN,NaN,16.15,NaN,NaN,NaN
227,316013711,2017-01-01T00:01:13,48.43147,-123.37628,0.0,20.7,511.0,CFAV GLENDYNE,NaN,CGNF,NaN,under way using engine,29.00,NaN,NaN,NaN
235,316024153,2017-01-01T00:01:18,49.29179,-123.12605,0.0,-198.9,511.0,LADY SUZANNE,NaN,NaN,NaN,NaN,13.38,4.50,1.9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20501764,367553550,2017-01-31T23:47:55,47.24466,-122.55770,0.0,-68.8,511.0,ENDEAVOR,NaN,WDG6105,NaN,under way using engine,11.89,3.96,NaN,NaN
20501827,316028292,2017-01-31T23:45:40,49.19933,-123.13010,7.1,-158.1,511.0,RIVER REBEL,NaN,NaN,NaN,under way using engine,10.30,4.88,1.1,NaN
20501830,367704010,2017-01-31T23:46:54,45.70463,-122.86770,1.3,19.0,511.0,TWO SMOOTH,NaN,WDI4985,NaN,NaN,11.28,NaN,NaN,NaN
20501832,316028292,2017-01-31T23:51:31,49.20516,-123.15115,10.1,-93.8,511.0,RIVER REBEL,NaN,NaN,NaN,under way using engine,10.30,4.88,1.1,NaN


In [10]:
vessel_group = {}
for t in ves_type:
    if 60 <= t <= 69 or 1012 <= t <= 1015: vessel_group[t] = 'Passenger'
    elif 70 <= t <= 79 or 1003 <= t <= 1004 or t == 1016: vessel_group[t] = 'Cargo'
    elif 80 <= t <= 89 or t == 1017 or t == 1024: vessel_group[t] = 'Tanker'
    elif t == 1018 or t == 1020 or t == 1022: vessel_group[t] = 'Other'
    elif 36 <= t <= 37 or t == 1019: vessel_group[t] = 'Pleasure Craft'
    elif t == 35 or t == 1021: vessel_group[t] = 'Military'
    elif 21 <= t <= 22 or 31 <= t <= 32 or t == 52 or t == 1023 or t == 1025: vessel_group[t] = 'Tug Tow'
    elif t == 30 or 1001 <= t < 1002: vessel_group[t] = 'Fishing'

NameError: name 'ves_type' is not defined

In [15]:
vessel_group

{30: 'Fishing',
 31: 'Tug Tow',
 32: 'Tug Tow',
 35: 'Military',
 37: 'Pleasure Craft',
 52: 'Tug Tow',
 60: 'Passenger',
 70: 'Cargo',
 71: 'Cargo',
 74: 'Cargo',
 79: 'Cargo',
 80: 'Tanker',
 1001: 'Fishing',
 1003: 'Cargo',
 1004: 'Cargo',
 1012: 'Passenger',
 1013: 'Passenger',
 1017: 'Tanker',
 1018: 'Other',
 1019: 'Pleasure Craft',
 1020: 'Other',
 1022: 'Other',
 1023: 'Tug Tow',
 1024: 'Tanker',
 1025: 'Tug Tow'}

In [4]:
month_list = []
for i in range(1, 13):
    if i < 10:
        month_list.append('0'+str(i))
    else:
        month_list.append(str(i))

In [4]:
vessel_group = {}

In [10]:
for m in month_list:
    data = pd.read_csv(f'./data/fuca/fuca_{m}.csv')
    ves_type = list(sorted(data['VesselType'].unique()))
    ves_type = [x for x in ves_type if pd.isnull(x) == False]
    ves_type = list(map(int, ves_type))
    for t in ves_type:
        if 60 <= t <= 69 or 1012 <= t <= 1015: vessel_group[t] = 'Passenger'
        elif 70 <= t <= 79 or 1003 <= t <= 1004 or t == 1016: vessel_group[t] = 'Cargo'
        elif 80 <= t <= 89 or t == 1017 or t == 1024: vessel_group[t] = 'Tanker'
        elif t == 1018 or t == 1020 or t == 1022: vessel_group[t] = 'Other'
        elif 36 <= t <= 37 or t == 1019: vessel_group[t] = 'Pleasure Craft'
        elif t == 35 or t == 1021: vessel_group[t] = 'Military'
        elif 21 <= t <= 22 or 31 <= t <= 32 or t == 52 or t == 1023 or t == 1025: vessel_group[t] = 'Tug Tow'
        elif t == 30 or 1001 <= t < 1002: vessel_group[t] = 'Fishing'
del data

In [11]:
vessel_group

{30: 'Fishing',
 31: 'Tug Tow',
 32: 'Tug Tow',
 35: 'Military',
 37: 'Pleasure Craft',
 52: 'Tug Tow',
 60: 'Passenger',
 70: 'Cargo',
 71: 'Cargo',
 74: 'Cargo',
 79: 'Cargo',
 80: 'Tanker',
 1001: 'Fishing',
 1003: 'Cargo',
 1004: 'Cargo',
 1012: 'Passenger',
 1013: 'Passenger',
 1017: 'Tanker',
 1018: 'Other',
 1019: 'Pleasure Craft',
 1020: 'Other',
 1022: 'Other',
 1023: 'Tug Tow',
 1024: 'Tanker',
 1025: 'Tug Tow',
 89: 'Tanker',
 1016: 'Cargo',
 76: 'Cargo',
 82: 'Tanker',
 36: 'Pleasure Craft'}

In [3]:
import pickle

In [13]:
with open('./data/vessel_type.pkl', 'wb') as f:
    pickle.dump(vessel_group, f)

In [11]:
with open('./data/vessel_type.pkl', 'rb') as f:
    vessel_group = pickle.load(f)

In [5]:
Fish = []
Tug = []
Military = []
Pleasure = []
Passenger = []
Cargo = []
Tanker = []
Other = []
for k, v in vessel_group.items():
    if v == 'Fishing': Fish.append(k)
    elif v == 'Tug Tow': Tug.append(k)
    elif v == 'Military': Military.append(k)
    elif v == 'Pleasure Craft': Pleasure.append(k)
    elif v == 'Passenger': Passenger.append(k)
    elif v == 'Cargo': Cargo.append(k)
    elif v == 'Tanker': Tanker.append(k)
    elif v == 'Other': Other.append(k)

In [24]:
data = pd.read_csv(f'./data/fuca/fuca_01.csv')

In [3]:
import pandas as pd
import numpy as np
import folium
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta
from tqdm import tqdm
import pickle

In [4]:
with open('./data/vessel_type.pkl', 'rb') as f:
    vessel_group = pickle.load(f)

In [5]:
month_list = []
for i in range(1, 13):
    if i < 10:
        month_list.append('0'+str(i))
    else:
        month_list.append(str(i))
Fish = []
Tug = []
Military = []
Pleasure = []
Passenger = []
Cargo = []
Tanker = []
Other = []
for k, v in vessel_group.items():
    if v == 'Fishing': Fish.append(k)
    elif v == 'Tug Tow': Tug.append(k)
    elif v == 'Military': Military.append(k)
    elif v == 'Pleasure Craft': Pleasure.append(k)
    elif v == 'Passenger': Passenger.append(k)
    elif v == 'Cargo': Cargo.append(k)
    elif v == 'Tanker': Tanker.append(k)
    elif v == 'Other': Other.append(k)

In [8]:
cargo_df_list = []

for m in month_list[6:]:
    data = pd.read_csv(f'./data/fuca/fuca_{m}.csv')
    is_in = data['VesselType'].isin(Cargo)
    temp_df = data[is_in]
    cargo_df_list.append(temp_df)
    print(f"{m} done")
del data

cargo_df = pd.concat(cargo_df_list)
# fuca_tug.to_csv(f'./data/fuca/fuca_tug.csv', index=False)

07 done
08 done
09 done
10 done
11 done
12 done


In [9]:
cargo_df.to_csv(f'./data/fuca/fuca_cargo_2.csv', index=False)

In [7]:
del military_df

In [7]:
data = pd.read_csv(f'./data/fuca/fuca_12.csv')
is_in = data['VesselType'].isin(Fish)
temp_df = data[is_in]
fish_fuca = fish_fuca.append(temp_df)

/tmp/ipykernel_14599/4217968854.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fish_fuca = fish_fuca.append(temp_df)


In [8]:
fish_fuca.to_csv(f'./data/fuca/fuca_fish.csv', index=False)

In [6]:
del fish_fuca

NameError: name 'fish_fuca' is not defined

In [10]:
for m in month_list:
    data = pd.read_csv(f'./data/fuca/fuca_{m}.csv')
    mmsi = data['MMSI'].unique()
    if 369970121 in mmsi:
        print(f'369970121: {data[data["MMSI"] == 369970121]["VesselType"].iloc[0]}')
    if 376404000 in mmsi:
        print(f'376404000: {data[data["MMSI"] == 376404000]["VesselType"].iloc[0]}')
    if 369970121 in mmsi:
        print(f'369970121: {data[data["MMSI"] == 369970121]["VesselType"].iloc[0]}')
del data

369970121: 1018.0
369970121: 1018.0
369970121: 1018.0
369970121: 1018.0
369970121: 1018.0
369970121: 1018.0
369970121: 1018.0
369970121: 1018.0
369970121: 1018.0
369970121: 1018.0
369970121: 1018.0
376404000: 1020.0
369970121: 1018.0
369970121: 1018.0
376404000: 1020.0
369970121: 1018.0
369970121: 1018.0
376404000: 1020.0
369970121: 1018.0
369970121: 1018.0
376404000: 1020.0
369970121: 1018.0
369970121: 1018.0
369970121: 1018.0
369970121: 1018.0
369970121: 1018.0
369970121: 1018.0
369970121: 1018.0


In [2]:
data = pd.DataFrame()
f1 = pd.read_csv("./data/fuca/fuca_other_1.csv")
f2 = pd.read_csv("./data/fuca/fuca_other_2.csv")
data = pd.concat([f1, f2])
data.to_csv(f'./data/fuca/fuca_other.csv', index=False)

In [2]:
import os
next(os.walk('./image/fish/120/1h'))[1]

['367002210',
 '368085000',
 '316018771',
 '316016663',
 '316005771',
 '316023833',
 '367155110',
 '366904590',
 '316019447',
 '368038000',
 '367595030',
 '367672390',
 '366954430',
 '367569470',
 '316012953',
 '367085290',
 '367618000',
 '368293000',
 '367197230',
 '366207650',
 '316014646',
 '338132000',
 '367645170',
 '367060330',
 '316005725',
 '316018759',
 '316005444',
 '316003459',
 '367353590',
 '316007990',
 '367063680',
 '316003050',
 '338104000',
 '368380000',
 '367170860',
 '367550330',
 '316003447',
 '366738920',
 '316005724',
 '367039250',
 '316002915',
 '368032000',
 '338854000',
 '366980490',
 '316020869',
 '367662190',
 '367070360',
 '316001021',
 '367651850',
 '316005715',
 '316005622',
 '367553420',
 '316002974',
 '316003514',
 '367418650',
 '316005416',
 '366046000',
 '316001731',
 '367335420',
 '366392280',
 '367161930',
 '316001222',
 '367440310',
 '316022149',
 '316003768',
 '367056452',
 '368272000',
 '367324840',
 '316005717',
 '367150410',
 '367617080',
 '3160

In [7]:
data = pd.read_csv('./data/fuca/small_fuca_loitering.csv')

/tmp/ipykernel_17650/1259771680.py:1: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./data/fuca/small_fuca_loitering.csv')


In [8]:
data

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo
0,367144080,2017-01-24T16:05:51,47.55512,-122.64682,0.0,-141.0,182.0,800 MANHATTAN,NaN,WDD4387,31.0,undefined,40.0,8.0,0.0,31.0
1,367144080,2017-01-24T16:06:58,47.55512,-122.64680,0.0,-141.0,182.0,800 MANHATTAN,NaN,WDD4387,31.0,undefined,40.0,8.0,0.0,31.0
2,367144080,2017-01-24T16:15:33,47.55512,-122.64682,0.0,-141.0,182.0,800 MANHATTAN,NaN,WDD4387,31.0,undefined,40.0,8.0,0.0,31.0
3,367144080,2017-01-24T16:12:13,47.55512,-122.64680,0.0,-141.0,182.0,800 MANHATTAN,NaN,WDD4387,31.0,undefined,40.0,8.0,0.0,31.0
4,367144080,2017-01-24T16:10:03,47.55513,-122.64682,0.0,-141.0,182.0,800 MANHATTAN,NaN,WDD4387,31.0,undefined,40.0,8.0,0.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18873829,367455690,2017-10-11T17:14:45,46.14423,-123.86214,0.0,-96.8,511.0,GOOD PARTNER,NaN,WDF4773,1002.0,undefined,22.1,NaN,NaN,30.0
18873830,367455690,2017-10-11T17:20:02,46.14414,-123.86209,0.0,-104.1,511.0,GOOD PARTNER,NaN,WDF4773,1002.0,undefined,22.1,NaN,NaN,30.0
18873831,367455690,2017-10-11T17:23:43,46.14417,-123.86211,0.0,-87.1,511.0,GOOD PARTNER,NaN,WDF4773,1002.0,undefined,22.1,NaN,NaN,30.0
18873832,367455690,2017-10-11T17:28:13,46.14419,-123.86213,0.0,-79.3,511.0,GOOD PARTNER,NaN,WDF4773,1002.0,undefined,22.1,NaN,NaN,30.0


In [14]:
data[data['MMSI']==369970121]

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo
6952868,369970121,2017-04-19T17:55:44,46.19431,-123.83470,8.4,83.5,86.0,ELTON,NaN,AEOA,1018.0,restricted maneuverability,18.26,7.2,NaN,0.0
6952869,369970121,2017-04-19T18:02:05,46.19942,-123.81348,9.6,62.5,64.0,ELTON,NaN,AEOA,1018.0,restricted maneuverability,18.26,7.2,NaN,0.0
6952870,369970121,2017-04-19T18:05:17,46.20349,-123.80274,9.6,62.0,58.0,ELTON,NaN,AEOA,1018.0,restricted maneuverability,18.26,7.2,NaN,0.0
6952871,369970121,2017-04-19T18:03:13,46.20095,-123.80947,9.6,61.3,62.0,ELTON,NaN,AEOA,1018.0,restricted maneuverability,18.26,7.2,NaN,0.0
6952872,369970121,2017-04-19T17:59:56,46.19663,-123.82088,9.8,61.8,63.0,ELTON,NaN,AEOA,1018.0,restricted maneuverability,18.26,7.2,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6978479,369970121,2017-12-28T01:09:41,46.26999,-123.47825,13.1,-131.1,276.0,ELTON,NaN,AEOA,1018.0,restricted maneuverability,18.26,7.2,NaN,0.0
6978480,369970121,2017-12-28T01:10:43,46.27065,-123.48286,3.4,-94.2,50.0,ELTON,NaN,AEOA,1018.0,restricted maneuverability,18.26,7.2,NaN,0.0
6978481,369970121,2017-12-28T01:13:50,46.27176,-123.48189,9.3,92.9,99.0,ELTON,NaN,AEOA,1018.0,restricted maneuverability,18.26,7.2,NaN,0.0
6978482,369970121,2017-12-28T01:27:27,46.27014,-123.46924,14.0,-96.2,314.0,ELTON,NaN,AEOA,1018.0,restricted maneuverability,18.26,7.2,NaN,0.0


In [12]:
vessel_group

{30: 'Fishing',
 31: 'Tug Tow',
 32: 'Tug Tow',
 35: 'Military',
 37: 'Pleasure Craft',
 52: 'Tug Tow',
 60: 'Passenger',
 70: 'Cargo',
 71: 'Cargo',
 74: 'Cargo',
 79: 'Cargo',
 80: 'Tanker',
 1001: 'Fishing',
 1003: 'Cargo',
 1004: 'Cargo',
 1012: 'Passenger',
 1013: 'Passenger',
 1017: 'Tanker',
 1018: 'Other',
 1019: 'Pleasure Craft',
 1020: 'Other',
 1022: 'Other',
 1023: 'Tug Tow',
 1024: 'Tanker',
 1025: 'Tug Tow',
 89: 'Tanker',
 1016: 'Cargo',
 76: 'Cargo',
 82: 'Tanker',
 36: 'Pleasure Craft'}